In [1]:
import torch
from scipy.spatial import distance
import numpy as np
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


model = torch.load("model-final2", map_location=device)
model.to(device)
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [2]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('DeepPavlov/rubert-base-cased-sentence', model_max_length=512)


In [29]:
request = input()
token = tokenizer([request], truncation=True, padding=True)
input_ids = torch.tensor(token.input_ids).to(device)
attention_mask = torch.tensor(token.attention_mask).to(device)
outputs = model(input_ids, attention_mask=attention_mask, output_hidden_states = True)
label = np.argmax(outputs[0].detach().numpy(), 1)[0]
request_embedding = outputs.hidden_states[0][0][-1].detach().numpy()

Тактика оперативного лечения патологических переломов при миеломной болезни


In [30]:
from scipy.spatial import distance
import numpy as np
with open(str(label)+'.enc', 'r') as f:
    lines = f.readlines()
    sentences = lines[1::2]
    embeds = [np.fromstring(s, dtype=float, sep=' ') for s in lines[0::2]]
distances = []
for embed in embeds:
    distances.append(distance.cosine(embed, request_embedding))
answers = [sentences[i] for i in np.argpartition(distances, 10)[:10]]
for s, d in zip(answers, np.partition(distances, 10)[:10]):
    print(s+" "+str(d))
    

гистологического  риска,  стадия  III  Уровень  достоверности доказательств – 5)
 1.6698170401951984e-08
Уровень убедительности рекомендации А (уровень достоверности доказательств – 2)
 1.6698170401951984e-08
возможность  рассмотреть  переносить  способных  не  ** 3.1.3
 1.6698170401951984e-08
Уровень убедительности рекомендации C (уровень достоверности доказательств – 5)
 1.6698170401951984e-08
Уровень убедительности рекомендации А (уровень достоверности доказательств – 1)
 1.6698170401951984e-08
Уровень убедительности рекомендации А (уровень достоверности доказательств – 1)
 1.6698170401951984e-08
Уровень убедительности рекомендации С (уровень достоверности доказательств – 5)
 1.6698170401951984e-08
 При  появлении слабовыраженной  эритемы  дозу  облучения  оставляют  постоянной
 1.6698170401951984e-08
единиц»  для  определения  дозы  инсулина Уровень  достоверности доказательств – 5)
 1.6698170401951984e-08
Уровень убедительности рекомендации C (уровень достоверности доказательств –

In [31]:

import numpy as np
with open(str(label)+'.enc', 'r') as f:
    lines = f.readlines()
    sentences = lines[1::2]
    embeds = [np.fromstring(s, dtype=float, sep=' ') for s in lines[0::2]]
distances = []
# for embed in embeds:
score = np.sum(request_embedding * embeds, axis=1) / np.linalg.norm(embeds, axis=1)
topk_idx = np.argsort(score)[::-1][:10]
for idx in topk_idx:
    print('> %s\t%s' % (score[idx], sentences[idx]))

> 13.56550538280665	Диагноз подтверждается на основании обнаружения острого ИМ на аутопсии

> 13.56550538280665	 У  12-14%  пациентов  спленэктомия  неэффективна

> 13.56550538280665	 в  приложении  А3)  для предотвращения роста и метастазирования опухоли

> 13.56550538280665	Уровень убедительности рекомендаций IIa (уровень достоверности доказательств В)

> 13.56550538280665	-          выраженный кальциноз плечевой артерии, особенно у пожилых пациентов

> 13.56550538280665	 Прием кортикостероидов  был  разрешен,  но  не  являлся  обязательным

> 13.56550538280665	 Гемотрансфузии  проводятся  при  декомпенсации, связанной  гемоглобина

> 13.56550538280665	Уровень убедительности рекомендаций C (Уровень достоверности доказательств –5)

> 13.56550538280665	Комментарии: При ТУР МП удаляют все видимые опухоли

> 13.56550538280665	Лечение 3.1 Консервативное лечение 3.1.1



In [ ]:
label

In [17]:
import spacy
from spacy import displacy
nlp = spacy.load('ru_core_news_sm')
doc = nlp(request)

displacy.render(doc, jupyter=True, style="dep")